In [3]:
# opened w closed bas bey glitch
import cv2
from scipy.spatial import distance as dist
from imutils.video import VideoStream
import numpy as np
import imutils
import time

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def detect_eye_status(eye):
    # Apply Canny edge detection
    edges = cv2.Canny(eye, 30, 70)

    # Apply thresholding to create a binary image
    _, thresh = cv2.threshold(edges, 30, 255, cv2.THRESH_BINARY)

    # Apply morphological operations to remove noise and smooth the image
    kernel = np.ones((3, 3), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)

    # Find contours in the processed image
    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # If there are contours, the eyes are considered open
    if contours:
        return "Open"
    else:
        return "Closed"

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Constants for eye blink detection
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3

# Initialize variables for eye blink detection
COUNTER = 0
TOTAL = 0

# Open a connection to the camera (0 represents the default camera)
vs = VideoStream(src=0).start()
time.sleep(1.0)

while True:
    # Capture frame-by-frame
    frame = vs.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the region of interest for eye blink detection
        roi_gray = gray[y:y + h, x:x + w]

        # Extract the left and right eye coordinates
        leftEye = roi_gray[5:11, 0:w]
        rightEye = roi_gray[5:11, w - 6:w]

        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        ear = (leftEAR + rightEAR) / 2.0

        # Check for blinks
        if ear < EYE_AR_THRESH:
            COUNTER += 1
        else:
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                TOTAL += 1
            COUNTER = 0

        # Detect eye status using Canny edge detection, thresholding, and morphology
        left_eye_status = detect_eye_status(leftEye)
        right_eye_status = detect_eye_status(rightEye)

        # Draw contours for the eyes inside the detected face region
        leftEyeHull = cv2.convexHull(np.array([(30, 100), (30, 60), (w-30, 100), (w-30, 60)]))
        rightEyeHull = cv2.convexHull(np.array([(30, 100), (30, 60), (w-30, 100), (w-30, 60)]))
        # Draw green contours inside the blue rectangles for the eyes
        cv2.drawContours(frame[y:y + h, x:x + w], [leftEyeHull], -1, (0, 255, 0), 2)
        cv2.drawContours(frame[y:y + h, x:x + w], [rightEyeHull], -1, (0, 255, 0), 2)

        # Display eye status on the frame
        cv2.putText(frame, "Left Eye: {}".format(left_eye_status), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, "Right Eye: {}".format(right_eye_status), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

#         cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
#         cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('Face and Eye Detection', frame)

    # Break the loop if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release the camera and close the window
vs.stop()
cv2.destroyAllWindows()


In [2]:
import cv2
from scipy.spatial import distance as dist
from imutils.video import VideoStream
import numpy as np
import imutils
import time

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def detect_eye_status(left_eye, right_eye):
    # Calculate eye aspect ratios
    leftEAR = eye_aspect_ratio(left_eye)
    rightEAR = eye_aspect_ratio(right_eye)

    # Compare eye aspect ratios to check if they have similar relations
    ear_threshold = 0.05  # Adjust the threshold as needed
    ear_difference = abs(leftEAR - rightEAR)
    
    if ear_difference < ear_threshold:
        # Analyze color information to check if eyes are bright
        left_eye_mean_color = np.mean(left_eye)
        right_eye_mean_color = np.mean(right_eye)
        brightness_threshold = 150  # Adjust the threshold as needed

        if left_eye_mean_color > brightness_threshold and right_eye_mean_color > brightness_threshold:
            return "Awake"
        else:
            return "Closed"
    else:
        return "Closed"

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Constants for eye blink detection
EYE_AR_CONSEC_FRAMES = 3

# Initialize variables for eye blink detection
COUNTER = 0
TOTAL = 0

# Open a connection to the camera (0 represents the default camera)
vs = VideoStream(src=0).start()
time.sleep(1.0)

while True:
    # Capture frame-by-frame
    frame = vs.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the region of interest for eye blink detection
        roi_gray = gray[y:y + h, x:x + w]

        # Extract the left and right eye coordinates
        leftEye = roi_gray[5:11, 0:w]
        rightEye = roi_gray[5:11, w - 6:w]

        # Apply adaptive histogram equalization
        leftEye = cv2.equalizeHist(leftEye)
        rightEye = cv2.equalizeHist(rightEye)

        # Check for blinks
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        if (leftEAR + rightEAR) / 2 < EYE_AR_CONSEC_FRAMES:
            COUNTER += 1
        else:
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                TOTAL += 1
            COUNTER = 0

        # Detect eye status using EAR and color information
        eye_status = detect_eye_status(leftEye, rightEye)

        # Draw contours for the eyes inside the detected face region
        leftEyeHull = cv2.convexHull(np.array([(30, 100), (30, 60), (w-30, 100), (w-30, 60)]))
        rightEyeHull = cv2.convexHull(np.array([(30, 100), (30, 60), (w-30, 100), (w-30, 60)]))

        # Draw green contours inside the blue rectangles for the eyes
        cv2.drawContours(frame[y:y + h, x:x + w], [leftEyeHull], -1, (0, 255, 0), 2)
        cv2.drawContours(frame[y:y + h, x:x + w], [rightEyeHull], -1, (0, 255, 0), 2)

        # Display eye status on the frame
        cv2.putText(frame, "Eye Status: {}".format(eye_status), (x + 10, y + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
#         cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
#         cv2.putText(frame, "EAR: {:.2f}".format((leftEAR + rightEAR) / 2), (x + 10, y + 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('Face and Eye Detection', frame)

    # Break the loop if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release the camera and close the window
vs.stop()
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np
from scipy.spatial import distance as dist
from imutils.video import VideoStream
import imutils
import time

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def detect_eye_status(eye):
    # Apply Canny edge detection
    edges = cv2.Canny(eye, 30, 70)

    # Apply thresholding to create a binary image
    _, thresh = cv2.threshold(edges, 50, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Apply morphological operations to remove noise and smooth the image
    kernel = np.ones((3, 3), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)

    # Find contours in the processed image
    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # If there are contours, the eyes are considered open
    if contours:
        return "Open"
    else:
        return "Closed"

# Load the pre-trained face and eye detection models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Constants for eye blink detection
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3

# Initialize variables for eye blink detection
COUNTER = 0
TOTAL = 0

# Open a connection to the camera
vs = VideoStream(src=0).start()
time.sleep(1.0)

while True:
    # Capture frame-by-frame
    frame = vs.read()
    frame = imutils.resize(frame, width=450)

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the region of interest for the face
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]

        # Detect eyes within the face ROI
        eyes = eye_cascade.detectMultiScale(roi_gray)

        # Process each eye
        for (ex, ey, ew, eh) in eyes:
            eye_roi_gray = roi_gray[ey:ey + eh, ex:ex + ew]
            eye_roi_color = roi_color[ey:ey + eh, ex:ex + ew]

            # Calculate EAR and detect eye status
            eye_status = detect_eye_status(eye_roi_gray)
            
            # Draw green contours around the eyes
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

            # Display eye status on the frame
            cv2.putText(roi_color, eye_status, (ex, ey - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Face and Eye Detection', frame)

    # Break the loop if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release the camera and close the window
vs.stop()
cv2.destroyAllWindows()


In [23]:
import cv2
import numpy as np
from scipy.spatial import distance as dist
from imutils.video import VideoStream
import imutils
import time

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def detect_eye_status(eye):
    # Apply Canny edge detection
    edges = cv2.Canny(eye, 30, 70)

    # Apply thresholding to create a binary image
    _, thresh = cv2.threshold(edges, 30, 255, cv2.THRESH_BINARY)

    # Apply morphological operations to remove noise and smooth the image
    kernel = np.ones((3, 3), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)

    # Find contours in the processed image
    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # If there are contours, the eyes are considered open
    if contours:
        return "Open"
    else:
        return "Closed"

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Constants for eye blink detection
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3

# Initialize variables for eye blink detection
COUNTER = 0
TOTAL = 0

# Open a connection to the camera (0 represents the default camera)
vs = VideoStream(src=0).start()
time.sleep(1.0)

while True:
    # Capture frame-by-frame
    frame = vs.read()
    frame = imutils.resize(frame, width=450)

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the region of interest for eye blink detection
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]

        eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 10)

        for (ex, ey, ew, eh) in eyes:
            eye_roi_gray = roi_gray[ey:ey + eh, ex:ex + ew]
            eye_status = detect_eye_status(eye_roi_gray)

            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
            cv2.putText(roi_color, eye_status, (ex, ey - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Face and Eye Detection', frame)

    # Break the loop if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release the camera and close the window
vs.stop()
cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np
from imutils.video import VideoStream
import imutils
import time

def detect_eye_status(eye_gray):
    # Calculate the average pixel intensity in the eye region
    average_intensity = np.mean(eye_gray)

    # Threshold to determine open or closed state
    # You might need to adjust this threshold
    if average_intensity < 40:  # Threshold value, may need adjustment
        return "Closed"
    else:
        return "Open"

# Load the pre-trained face and eye detection models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Open a connection to the camera
vs = VideoStream(src=0).start()
time.sleep(1.0)

while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]

        eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 10)

        for (ex, ey, ew, eh) in eyes:
            eye_roi_gray = roi_gray[ey:ey + eh, ex:ex + ew]

            # Detect eye status (Open or Closed)
            eye_status = detect_eye_status(eye_roi_gray)

            # Draw rectangle around the eyes
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
            cv2.putText(roi_color, eye_status, (ex, ey - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow('Face and Eye Detection', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

vs.stop()
cv2.destroyAllWindows()


In [21]:
import cv2
import numpy as np
from imutils.video import VideoStream
import imutils

def is_eye_closed(eye):
    gray_eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
    blurred_eye = cv2.GaussianBlur(gray_eye, (7, 7), 0)
    
    # Use adaptive thresholding for more robust detection
    thresh_eye = cv2.adaptiveThreshold(blurred_eye, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    white_pixels = cv2.countNonZero(thresh_eye)
    white_area_percentage = (white_pixels / (thresh_eye.size)) * 100

    # Adjust the threshold based on experimental results
    return white_area_percentage > 30

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

vs = VideoStream(src=0).start()

while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        roi_color = frame[y:y + h, x:x + w]
        roi_gray = gray[y:y + h, x:x + w]

        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5)

        for (ex, ey, ew, eh) in eyes:
            eye = roi_color[ey:ey + eh, ex:ex + ew]
            eye_status = 'Closed' if is_eye_closed(eye) else 'Open'

            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
            cv2.putText(roi_color, eye_status, (ex, ey - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow('Eye Open or Closed Detection', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

vs.stop()
cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np
from imutils.video import VideoStream
import imutils

def is_eye_closed(eye):
    # Convert the eye region to grayscale for processing
    gray_eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to smooth the image and reduce noise, helping in contour detection
    blurred_eye = cv2.GaussianBlur(gray_eye, (7, 7), 0)

    # Histogram Equalization to improve the contrast of the image
    equalized_eye = cv2.equalizeHist(blurred_eye)

    # Apply adaptive thresholding to convert the image to binary (black and white)
    _, thresh_eye = cv2.threshold(equalized_eye, 70, 255, cv2.THRESH_BINARY_INV)

    # Find contours from the binary image - contours are used to count the number of white regions
    contours, _ = cv2.findContours(thresh_eye, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Determine if the eye is closed based on the number of white regions detected
    # Fewer contours typically indicate a closed eye
    return len(contours) < 2  # Threshold value might need adjustment for different conditions

# Load the Haar Cascade Classifiers for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Start video stream from the default camera
vs = VideoStream(src=0).start()

while True:
    # Read frame from the video stream and resize it
    frame = vs.read()
    frame = imutils.resize(frame, width=450)

    # Convert frame to grayscale to facilitate face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Loop over the face detections
    for (x, y, w, h) in faces:
        # Extract the region of interest (ROI) for the face in the frame
        roi_color = frame[y:y + h, x:x + w]
        roi_gray = gray[y:y + h, x:x + w]

        # Detect eyes within the face ROI
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5)

        # Loop over the eye detections
        for (ex, ey, ew, eh) in eyes:
            # Extract the eye region from the color face ROI
            eye = roi_color[ey:ey + eh, ex:ex + ew]

            # Determine the status of the eye (open or closed)
            closed = is_eye_closed(eye)
            eye_status = 'Closed' if closed else 'Open'

            # Draw a rectangle around the eye and label it with its status
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
            cv2.putText(roi_color, eye_status, (ex, ey - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display the resulting frame with detected faces and eyes
    cv2.imshow('Eye Open or Closed Detection', frame)

    # Break the loop if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release the video stream and close all windows
vs.stop()
cv2.destroyAllWindows()
